In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data = data[data["state"] == 6]

In [5]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100

In [6]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
#Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
#Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount
which_bank,,,,
All Other Banks,51.791350,61.556190,2.700681e+07,3.104069e+07
Bank of West,2.465273,3.490267,1.388032e+06,1.871864e+06


In [8]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications']
Quintiles_loan_table3

num_approved_loans  num_applications  \
which_bank      pct_minority_Quintiles                                         
All Other Banks 20                                   98601            116528   
                40                                  153481            182401   
                60                                   91986            109877   
                80                                   51394             61507   
                100                                  12858             14993   
Bank of West    20                                    2784              3962   
                40                                    3651              5105   
                60                                    1959              2765   
                80                                    1458              2111   
                100                                    406               580   

                                        sum_approved_loans  \
which_bank      pct_minority_Quintiles                       
All Other Banks 20                            5.863134e+10   
                40                            7.796394e+10   
                60                            4.465607e+10   
                80                            2.485676e+10   
                100                           6.812430e+09   
Bank of West    20                            1.833340e+09   
                40                            2.037395e+09   
                60                            9.482750e+08   
                80                            7.251200e+08   
                100                           2.314700e+08   

                                        sum_all_loan_amount  \
which_bank      pct_minority_Quintiles                        
All Other Banks 20                             6.714449e+10   
                40                             8.935768e+10   
                60                             5.138694e+10   
                80                             2.907064e+10   
                100                            7.763915e+09   
Bank of West    20                             2.478280e+09   
                40                             2.706635e+09   
                60                             1.277105e+09   
                80                             9.965150e+08   
                100                            3.302900e+08   

                                        loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                               
All Other Banks 20                                        0.241480   
                40                                        0.375884   
                60                                        0.225279   
                80                                        0.125867   
                100                                       0.031490   
Bank of West    20                                        0.271398   
                40                                        0.355917   
                60                                        0.190973   
                80                                        0.142133   
                100                                       0.039579   

                                        $loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                                
All Other Banks 20                                         0.275367   
                40                                         0.366164   
                60                                         0.209731   
                80                                         0.116742   
                100                                        0.031995   
Bank of West    20                                         0.317428   
                40                                         0.352759   
                60                                         0.164186   
                80        

In [9]:
data['pct_White'] = data['Tot.WhitePop']/data['Tot.Pop'] * 100
data['pct_Black'] = data['Tot.BlackPop ']/data['Tot.Pop'] * 100
data['pct_AmericIndian'] = data['Tot.AmericIndianPop']/data['Tot.Pop'] * 100
data['pct_Asian'] = data['Tot.AsianPop']/data['Tot.Pop'] * 100
data['pct_NativeHawaiianPacific'] = data['Tot.NativeHawaiianPacificPop']/data['Tot.Pop'] * 100
data['pct_OtherRaceAlone'] = data['Tot.OtherRaceAlonePop']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace'] = data['Tot.TwoOrMoreRace']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace(Some other race)'] = data['Tot.TwoOrMoreRace(Some other race)']/data['Tot.Pop'] * 100
data['pct_Hispanic/Latino'] = data['Tot.Hispanic/Latino']/data['Tot.Pop'] * 100
data['pct_NotHispanic/Latino'] = data['Tot.NotHispanic/Latino']/data['Tot.Pop'] * 100

In [10]:
data['major_minority'] = data[['pct_Black', 'pct_AmericIndian', 'pct_Asian','pct_NativeHawaiianPacific',
                                        'pct_OtherRaceAlone','pct_TwoOrMoreRace','pct_TwoOrMoreRace(Some other race)',
                                        'pct_Hispanic/Latino']].idxmax(axis=1)
data['major_minority'] = data['major_minority'].apply(lambda x: 'Black' if x=='pct_Black' 
                                                                        else 'American Indian' if x=='pct_AmericIndian' 
                                                                        else 'Asian' if x=='pct_Asian' 
                                                                        else 'Native Hawaiian Pacific' if x=='pct_NativeHawaiianPacific' 
                                                                        else 'Other Race Alone' if x=='pct_OtherRaceAlone' 
                                                                        else 'Two Or More Race' if x=='pct_TwoOrMoreRace' 
                                                                        else 'Two Or More Race(Some other race)' if x=='pct_TwoOrMoreRace(Some other race)' 
                                                                        else 'Hispanic/Latino')

In [11]:
Quintiles_loan_table4 = data.groupby(['which_bank', "major_minority"])[hmda_vars].sum()
Quintiles_loan_table4['loan_pct_in_minority_group'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['$loan_pct_in_minority_group'] = Quintiles_loan_table4['sum_approved_loans']/Quintiles_loan_table4['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['loan_approval_rate'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_applications']
Quintiles_loan_table4

num_approved_loans  num_applications  \
which_bank      major_minority                                                  
All Other Banks American Indian                         497               614   
                Asian                                114107            129526   
                Black                                  6913              8484   
                Hispanic/Latino                      282196            341292   
                Native Hawaiian Pacific                   9                10   
                Other Race Alone                        116               145   
                Two Or More Race                       4485              5238   
Bank of West    American Indian                           9                10   
                Asian                                  3903              5592   
                Black                                   183               247   
                Hispanic/Latino                        5951              8372   
                Other Race Alone                         12                19   
                Two Or More Race                        200               283   

                                         sum_approved_loans  \
which_bank      major_minority                                
All Other Banks American Indian                1.602250e+08   
                Asian                          7.782094e+10   
                Black                          3.519875e+09   
                Hispanic/Latino                1.266412e+11   
                Native Hawaiian Pacific        1.605000e+06   
                Other Race Alone               1.037400e+08   
                Two Or More Race               4.674075e+09   
Bank of West    American Indian                5.675000e+06   
                Asian                          2.810865e+09   
                Black                          8.808500e+07   
                Hispanic/Latino                2.668265e+09   
                Other Race Alone               6.390000e+06   
                Two Or More Race               1.963200e+08   

                                         sum_all_loan_amount  \
which_bank      major_minority                                 
All Other Banks American Indian                 1.914000e+08   
                Asian                           8.665058e+10   
                Black                           4.181220e+09   
                Hispanic/Latino                 1.482122e+11   
                Native Hawaiian Pacific         1.960000e+06   
                Other Race Alone                1.203850e+08   
                Two Or More Race                5.367100e+09   
Bank of West    American Indian                 5.870000e+06   
                Asian                           3.835540e+09   
                Black                           1.136550e+08   
                Hispanic/Latino                 3.552510e+09   
                Other Race Alone                8.395000e+06   
                Two Or More Race                2.728550e+08   

                                         loan_pct_in_minority_group  \
which_bank      major_minority                                        
All Other Banks American Indian                            0.001217   
                Asian                                      0.279453   
                Black                                      0.016930   
                Hispanic/Latino                            0.691110   
                Native Hawaiian Pacific                    0.000022   
                Other Race Alone                           0.000284   
                Two Or More Race                           0.010984   
Bank of West    American Indian                            0.000877   
                Asian                                      0.380484   
                Black                                      0.017840   
                Hispanic/Latino                            0.58013